In [35]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib import font_manager, rc

try :
    import folium
    from folium.plugins import HeatMap
except :
    !pip install folium
    import folium
    from folium.plugins import HeatMap

# 한글 폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # Windows의 '맑은 고딕' 폰트 경로
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [37]:
# 데이터 불러오기
gwangjin = pd.read_csv('./data/make_file/광진구_모든_시설의_위치.csv', index_col=0)

In [39]:

# 지도 중심 좌표 설정 (광진구 내 평균 좌표)
map_center = [gwangjin['위도'].mean(), gwangjin['경도'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 히트맵 추가
heat_data = gwangjin[['위도', '경도']].values.tolist()
HeatMap(heat_data, radius=10).add_to(m)

# 지도 출력
m
